In [ ]:
import requests
import defs
import pandas as pd
import datetime

Open a session so that requests don't have to re-connect each time

In [ ]:
session = requests.Session()

Consider turning instrument(s) into a list...

In [ ]:
ins_df = pd.read_csv("./Data/available_instruments.csv").drop(["Unnamed: 0"], axis=1)
ins_df.head()

In [ ]:
our_curr = ["GBP", "USD", "EUR", "JPY", "CHF", "NZD", "CAD"]

In [ ]:
def fetch_candles(pair_name, count, granularity):
    url = f"{defs.OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price="MBA"
    )
    response = session.get(url, params=params, headers=defs.SECURE_HEADER)
    return response.status_code, response.json()

In [ ]:

status_code, res = fetch_candles("EUR_USD", 10, "H1")
print(status_code)

In [ ]:
def get_candles_df(response):

    our_data =[]

    prices = ["bid", "mid", "ask"]
    ohlc = ["o", "h", "l", "c"]

    for candle in response["candles"]:
        if candle["complete"] == False:
            continue

        new_dict = {}
        new_dict["ticker"] = response["instrument"]
        new_dict["time"] = candle["time"]
        new_dict["volume"] = candle["volume"]
        
        for price in prices:
            for oh in ohlc:
                new_dict[f"{price}_{oh}"] = candle[price][oh]

        our_data.append(new_dict)
        
    return pd.DataFrame.from_dict(our_data)


In [ ]:
df = get_candles_df(res)
df

In [ ]:
def save_file(candles_df, pair, granularity):
    # candles_df.to_csv(f"./data/{pair}_{granularity} {datetime.datetime.now().strftime('%Y%m%d%H%M')}.csv")
    candles_df.to_csv(f"./data/{pair}_{granularity}.csv")

In [ ]:
save_file(df, "EUR_USD", "H1")

In [ ]:
def create_data(pair, granularity):
    code, json_data = fetch_candles(pair, 4000, granularity)
    if code != 200:
        print(f"{pair} Error: {code}")
        return
    df = get_candles_df(json_data)
    print(f"{pair} loaded {df.shape[0]} candles from {df.time.min()} to {df.time.max()}")
    save_file(df, pair, granularity)

In [ ]:
create_data("EUR_USD", "H1")

In [19]:
def our_pairs(our_curr):

    pairs=[]
    
    for i in range(len(our_curr)):
        if i== 0:
            j=1
        else:
            j=0

        while (j) < len(our_curr):
            if i != j:
                new_pair = f"{our_curr[i]}_{our_curr[j]}"
                if new_pair in ins_df.name.unique():
                    pairs.append(f"{our_curr[i]}_{our_curr[j]}")
            j = j + 1
    
    return pairs


In [23]:
pairs = our_pairs(our_curr)
pairs


['GBP_USD',
 'GBP_JPY',
 'GBP_CHF',
 'GBP_NZD',
 'GBP_CAD',
 'USD_JPY',
 'USD_CHF',
 'USD_CAD',
 'EUR_GBP',
 'EUR_USD',
 'EUR_JPY',
 'EUR_CHF',
 'EUR_NZD',
 'EUR_CAD',
 'CHF_JPY',
 'NZD_USD',
 'NZD_JPY',
 'NZD_CHF',
 'NZD_CAD',
 'CAD_JPY',
 'CAD_CHF']

In [24]:
for pair in pairs:
    create_data(pair, "H1")

GBP_USD loaded 3999 candles from 2022-11-22T20:00:00.000000000Z to 2023-07-14T10:00:00.000000000Z
GBP_JPY loaded 3999 candles from 2022-11-22T20:00:00.000000000Z to 2023-07-14T10:00:00.000000000Z
GBP_CHF loaded 3999 candles from 2022-11-22T20:00:00.000000000Z to 2023-07-14T10:00:00.000000000Z
GBP_NZD loaded 3999 candles from 2022-11-22T20:00:00.000000000Z to 2023-07-14T10:00:00.000000000Z
GBP_CAD loaded 3999 candles from 2022-11-22T20:00:00.000000000Z to 2023-07-14T10:00:00.000000000Z
USD_JPY loaded 3999 candles from 2022-11-22T20:00:00.000000000Z to 2023-07-14T10:00:00.000000000Z
USD_CHF loaded 3999 candles from 2022-11-22T20:00:00.000000000Z to 2023-07-14T10:00:00.000000000Z
USD_CAD loaded 3999 candles from 2022-11-22T20:00:00.000000000Z to 2023-07-14T10:00:00.000000000Z
EUR_GBP loaded 3999 candles from 2022-11-22T20:00:00.000000000Z to 2023-07-14T10:00:00.000000000Z
EUR_USD loaded 3999 candles from 2022-11-22T20:00:00.000000000Z to 2023-07-14T10:00:00.000000000Z
EUR_JPY loaded 3999 

In [ ]:
for pair in pairs:
    create_data(pair, "H1")

In [ ]:
import matplotlib.pyplot as plt

testdf = pd.read_csv("./Data/CAD_CHF_H1 202307071009.csv")
testdf["bid_o"].plot()